In [2]:
from textSummarizer.logging import logger

In [3]:
import os
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\riswa\\Desktop\\Text-Summarizer-Project'

# entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [24]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path
    tokenizer_name: str
    

#$ configuration  manager

In [25]:
from src.textSummarizer.utils.common import read_yaml,create_directories
from src.textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [26]:
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config['data_transformation']
        create_directories([Path(config['root_dir'])])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config['root_dir']),
            data_path=Path(config['data_path']),
            tokenizer_name=config['tokenizer_name']  # This remains a string
        )
        return data_transformation_config


In [27]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

## Components

In [28]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
from pathlib import Path

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e


[2024-07-09 20:53:21,661: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-09 20:53:21,668: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 20:53:21,673: INFO: common: artifacts have been created]
[2024-07-09 20:53:21,675: INFO: common: created directory at: artifacts]
[2024-07-09 20:53:21,679: INFO: common: artifacts\data_transformation have been created]
[2024-07-09 20:53:21,680: INFO: common: created directory at: artifacts\data_transformation]


c:\Users\riswa\anaconda3\envs\text_summarizer\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\riswa\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\riswa\anaconda3\envs\text_su

In [20]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-07-09 20:51:17,906: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-09 20:51:17,914: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 20:51:17,914: INFO: common: artifacts have been created]
[2024-07-09 20:51:17,914: INFO: common: created directory at: artifacts]
[2024-07-09 20:51:17,914: INFO: common: artifacts/data_transformation have been created]
[2024-07-09 20:51:17,922: INFO: common: created directory at: artifacts/data_transformation]


OSError: Incorrect path_or_model_id: 'google\pegasus-cnn_dailymail'. Please provide either the path to a local folder or the repo_id of a model on the Hub.